CNN,ensemble을 class를 이용해서 구현해보자.

class 객체를 만들어서 각 개체에 CNN 모델을 하나씩 저장하는 방식으로 ensemble cnn 학습을 진행한다.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# 모듈 삽입

import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [0]:
# kaggle 데이터

train_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/co_data/digit-recognizer/train.csv')
test_data = np.array(pd.read_csv('/content/drive/My Drive/Colab Notebooks/co_data/digit-recognizer/test.csv'))

x_data = np.array(train_data.drop(['label'], axis='columns', inplace=False))
y_data = np.array(train_data['label'])
y_data = pd.get_dummies(train_data.label).values
scaler=MinMaxScaler()
x_data = scaler.fit_transform(x_data)
        
split_num = int(x_data.shape[0]*0.8)
train_x_data = x_data[:split_num]
test_x_data = x_data[split_num:]
train_y_data = y_data[:split_num]
test_y_data = y_data[split_num:]

In [0]:
# 그래프 초기화
tf.reset_default_graph()

# 객체지향

class CnnModel:
  
    def __init__(self, session, train_x_data, train_y_data, test_data):
        print("객체가 생성되었어요!")
        # class 불러오면 바로 실행되는 부분
        # tf 그래프 그리기 > 학습 > 예측 크게 세가지의 내용이 들어가도록 디자인하기
        # 바로 그래프 만들도록 연결!
        self.build_graph()
        # 괄호 안에 self?
        self.sess = session
        self.train_x_data = train_x_data
        self.train_y_data = train_y_data
        self.test_data = test_data
        # session은 입력 받아서 쓸 것, 모두 동일하게 사용
    
    def build_graph(self):
        print("텐서플로우 그래프를 그려요!")
        
        # self가 붙지 않은 변수들은 해당 함수 내에 국한이 된다: 지역변수
        # 같은 class 라고 하더라도 다른 함수라면 호출이 안됨
        # 따라서 다른 def 에서 사용할 변수는 self를 붙여줘야 함 
        
        self.X = tf.placeholder(shape=([None, 784]), dtype=tf.float32)
        self.Y = tf.placeholder(shape=([None, 10]), dtype=tf.float32)
        self.drop_rate = tf.placeholder(tf.float32)

        x_img = tf.reshape(self.X, [-1,28,28,1])
        
        L1 = tf.layers.conv2d(inputs = x_img,
                              filters = 32,
                              kernel_size = [3,3],
                              padding ="SAME",
                              strides = 1, 
                              activation = tf.nn.relu)
        L1 = tf.layers.max_pooling2d(inputs = L1,
                                     pool_size = [2,2],
                                     padding = "SAME",
                                     strides = 2)
        L2 = tf.layers.conv2d(inputs = L1,
                          filters = 64,
                          kernel_size = [3,3],
                          padding ="SAME",
                          strides = 1,
                          activation = tf.nn.relu)
        L2 = tf.layers.max_pooling2d(inputs = L2,
                                 pool_size = [2,2],
                                 padding = "SAME",
                                 strides = 2)
        
        convoled_data = tf.reshape(L2, [-1,7*7*64])

        layer1 = tf.layers.dense(inputs = convoled_data,
                                 units = 256,
                                 activation = tf.nn.relu)
        layer1 = tf.layers.dropout(inputs = layer1,
                                   rate = self.drop_rate)
        layer2 = tf.layers.dense(inputs = layer1,
                                 units = 128,   
                                 activation = tf.nn.relu)
        layer2 = tf.layers.dropout(inputs = layer2,
                                   rate = self.drop_rate)
        layer3 = tf.layers.dense(inputs = layer2,
                                 units = 512,  
                                 activation = tf.nn.relu)
        layer3 = tf.layers.dropout(inputs = layer2,
                                   rate = self.drop_rate)
        
        self.H = tf.layers.dense(inputs = layer3, units=10)
        self.cost = tf.losses.softmax_cross_entropy(self.Y, self.H)
        self.train = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(self.cost)
        
        
    def train_graph(self):
        print("텐서플로우 그래프를 학습시켜요!")
        self.sess.run(tf.global_variables_initializer())
        
        train_epoch = 50
        batch_size = 100
        
        for step in range(train_epoch):
            num_of_iter = int(self.train_x_data.shape[0] / batch_size)
            
            for i in range(num_of_iter):
                batch_x = self.train_x_data[i*batch_size:(i+1)*batch_size] 
                batch_y = self.train_y_data[i*batch_size:(i+1)*batch_size]
                
                _, cost_val = self.sess.run([self.train, self.cost],
                                            feed_dict={self.X: batch_x,
                                                       self.Y: batch_y,
                                                       self.drop_rate: 0.3})
            if step % 5 == 0:
                print(f'cost: {cost_val}')


    def H_graph_train(self):
        print("model 각각의 H 구해요!")
        self.H_val_train = sess.run(self.H, feed_dict={self.X: self.train_x_data,
                                                       self.drop_rate: 0})
        print(f'model 각각의 H: {self.H_val_train}')
        return (self.H_val_train)

    def ac_train(self):
        prediction = tf.argmax(self.H_val_train,1)
        correct = tf.equal(predict, tf.argmax(test_y_data,1))
        self.accuracy = tf.reduce_mean(correct)
        print(f'model 각각의 정확도: {self.accuracy}')
        return (self.accuracy)


    def H_graph_test(self):
        print("test data의 H 구해요!")
        self.H_val_test = sess.run(self.H,
                                   feed_dict={self.X: self.test_data,
                                              self.drop_rate: 0})
        return (self.H_val_test)





In [5]:
# 모델 10개를 만들 것 : 객체 10개를 만들 것
# CnnModel을 10번 호출하여 list 안에 넣음 

sess = tf.Session()
num_of_model = 10
models = [CnnModel(sess, train_x_data, train_y_data, test_data) for x in range(num_of_model)]

객체가 생성되었어요!
텐서플로우 그래프를 그려요!
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
객체가 생성되었어요!
텐서플로우 그래프를 그려요!
객체가 생성되었어요!
텐서플로우 그래프를 그려요!
객체가 생성되었어요!
텐서플로우 그래프를 그려요!
객체가 생성되었어요!
텐서플로우 그래프를 그려요!
객체가 생성되었어요!
텐서플로우 그래프를 그려요!
객체가 생성되었어요!
텐서플로우 그래프를 그려요!
객체가 생성되었어요!
텐서플로우 그래프를 그려요!
객체가 생성되었어요!
텐서플로우 그래프를 그려요!
객체가 생성되었어요!
텐서플로우 그래프를 그려요!


In [6]:
# models에 저장된 각 객체에 대해서 학습

for k in range(num_of_model):
    models[k].train_graph()

텐서플로우 그래프를 학습시켜요!
cost: 0.07725172489881516
cost: 0.008325785398483276
cost: 0.00016916710592340678
cost: 0.011305986903607845
cost: 0.00012987913214601576
cost: 0.0001335734996246174
cost: 1.9799907022388652e-05
cost: 1.5142570191528648e-05
cost: 3.910728082701098e-06
cost: 1.3505772358257673e-06
텐서플로우 그래프를 학습시켜요!
cost: 0.08851568400859833
cost: 0.01309104636311531
cost: 0.011005410924553871
cost: 0.05440745875239372
cost: 0.00025083572836592793
cost: 0.00046836453839205205
cost: 3.9916041714604944e-05
cost: 9.215622412739322e-06
cost: 3.0873661671648733e-06
cost: 1.1110128070868086e-06
텐서플로우 그래프를 학습시켜요!
cost: 0.07263101637363434
cost: 0.004021576140075922
cost: 0.002197176916524768
cost: 0.00014866476703900844
cost: 0.0010881343623623252
cost: 9.689400758361444e-05
cost: 0.00017170674982480705
cost: 3.7846964460186427e-06
cost: 1.028761744237272e-06
cost: 2.861004873011552e-07
텐서플로우 그래프를 학습시켜요!
cost: 0.078209288418293
cost: 0.024581406265497208
cost: 0.0012147427769377828
cost: 0.018

In [0]:
# 모든 모델에 prediction 데이터를 넣어 H 값을 모두 더한 argmax(H_result,1)fh 예측 값을 출력하는 것?

# 각각의 model에 대한 정확도를 구하는 것? 아니면 모든 모델이 종합(?)된 것에 대한 정확도를 구하는 것?

In [7]:
# train data 사용 모든 H 더한 값 구하기

H_result_train = np.zeros([])
for k in range(num_of_model):
    each_H_train = models[k].H_graph_train()
    H_result_train = np.add(H_result_train,each_H_train)
print(f'모든 H 더한 것: {H_result_train}')

model 각각의 H 구해요!
model 각각의 H: [[ 0.10533847  0.00676176 -0.00650846 ...  0.02539467  0.07369006
  -0.0142984 ]
 [ 0.2035768   0.06598719 -0.02684266 ... -0.08976451  0.0644847
   0.02755626]
 [ 0.03057863 -0.01978524 -0.03829113 ... -0.03637114  0.07668865
  -0.00671208]
 ...
 [ 0.12753025  0.0385867  -0.02478717 ... -0.10064307  0.11098446
   0.03863075]
 [ 0.1266085  -0.02473474 -0.01783815 ... -0.05387628  0.03380143
   0.0270507 ]
 [ 0.19266517  0.00751313  0.01839845 ... -0.02196606 -0.04909218
   0.00466025]]
model 각각의 H 구해요!
model 각각의 H: [[ 0.02486325 -0.01993037  0.01120128 ...  0.04379386 -0.02141951
  -0.02585897]
 [-0.02998987 -0.0244903   0.04990851 ...  0.07359132 -0.01484663
   0.00442363]
 [-0.03941043 -0.06766896 -0.00527938 ...  0.01608443 -0.01748767
   0.00146386]
 ...
 [ 0.0241329  -0.05385047  0.08346534 ...  0.04214837 -0.01240005
  -0.00536356]
 [-0.0371807  -0.03926507  0.00980386 ... -0.00668142  0.05757775
   0.04100674]
 [ 0.00083235 -0.04275719  0.11338097 .

In [8]:
# 제출용 data 사용 모든 H 더한 값 구하기

H_result_test = np.zeros([])
for k in range(num_of_model):
    each_H_test = models[k].H_graph_test()
    H_result_test = np.add(H_result_test,each_H_test)
print(f'모든 H 더한 것: {H_result_test}')

final = np.argmax(H_result_test,1)


test data의 H 구해요!
test data의 H 구해요!
test data의 H 구해요!
test data의 H 구해요!
test data의 H 구해요!
test data의 H 구해요!
test data의 H 구해요!
test data의 H 구해요!
test data의 H 구해요!
test data의 H 구해요!
모든 H 더한 것: [[-2703.2275 -5388.6562 14075.79   ... -4878.674  -2917.5295 -1614.1365]
 [ 9983.609  -4847.9575 -2068.874  ... -1323.667  -3633.568  -1920.5054]
 [-6237.965  -8413.302  -3691.8687 ... -4460.1533 -1239.8792  9429.101 ]
 ...
 [-3910.0803 -2486.3247 -4754.0674 ... -3094.1506 -2311.606  -3362.4626]
 [-5755.835  -6353.8047 -4337.276  ... -2191.2922 -2392.7642  7858.859 ]
 [-3916.6838 -4785.1807 13270.716  ... -4761.8486 -1015.0341 -1869.9313]]


In [9]:
submission = pd.read_csv('/content/drive/My Drive/Colab Notebooks/co_data/digit-recognizer/sample_submission.csv')
final_result= pd.DataFrame({'Label':final}) 
submission['Label'] = final_result['Label']
submission.to_csv('/content/drive/My Drive/Colab Notebooks/co_data/submission.csv', index=False)
submission

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9
